# EMA Crossover Trading Strategy with Regime Filter

This notebook implements a simple EMA-based trading strategy.
Market regimes detected using HMM are used as a filter to improve trade quality.


In [7]:
import pandas as pd
import numpy as np


In [8]:
features = pd.read_csv("../data/nifty_features_5min.csv")
features["timestamp"] = pd.to_datetime(features["timestamp"])
features.head()
# features.columns


,timestamp,open,high,low,close,volume,ema_5,ema_15,spot_return,futures_basis,avg_iv,iv_spread,pcr_oi,pcr_volume,regime,market_regime
0,2015-01-09 09:20:00,8300.50,8303.00,8293.25,8301.00,0,8301.133333,8301.175000,-0.000024,0.0005,0.209528,-0.001660,1.215374,0.941080,2,1
1,2015-01-09 09:25:00,8301.65,8302.55,8286.80,8294.15,0,8298.805556,8300.296875,-0.000826,0.0005,0.187823,-0.012707,3.066087,0.621079,2,1
2,2015-01-09 09:30:00,8294.10,8295.75,8280.65,8288.50,0,8295.370370,8298.822266,-0.000681,0.0005,0.191463,-0.024234,0.516905,0.395105,2,1
3,2015-01-09 09:35:00,8289.10,8290.45,8278.00,8283.45,0,8291.396914,8296.900732,-0.000609,0.0005,0.173081,0.045038,2.092148,4.619883,1,-1
4,2015-01-09 09:40:00,8283.40,8288.30,8277.40,8285.55,0,8289.447942,8295.481891,0.000253,0.0005,0.225163,-0.020501,2.222767,0.645333,2,1


## Trading Signals

Trading signals are generated using EMA(5) and EMA(15) crossover logic.


In [9]:
features["signal"] = 0

features.loc[
    (features["ema_5"] > features["ema_15"]),
    "signal"
] = 1

features.loc[
    (features["ema_5"] < features["ema_15"]),
    "signal"
] = -1


## Regime-Based Filtering

Trades are allowed only when the trading signal agrees with the detected market regime.


In [10]:
features["filtered_signal"] = 0

features.loc[
    (features["signal"] == 1) & (features["market_regime"] == 1),
    "filtered_signal"
] = 1

features.loc[
    (features["signal"] == -1) & (features["market_regime"] == -1),
    "filtered_signal"
] = -1


## Strategy Backtesting

A simple next-candle execution assumption is used.
Transaction costs and slippage are ignored for simplicity.


In [11]:
features["strategy_return"] = (
    features["filtered_signal"].shift(1) * features["spot_return"]
)
# Save features with strategy returns
features.to_csv("../data/nifty_features_5min.csv", index=False)


features["strategy_return"].fillna(0, inplace=True)


C:\Users\navin\AppData\Local\Temp\ipykernel_18908\1926691071.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  features["strategy_return"].fillna(0, inplace=True)


## Performance Metrics


In [12]:
def calculate_metrics(returns):
    total_return = returns.sum()
    sharpe = np.sqrt(252 * 78) * returns.mean() / returns.std()
    max_drawdown = (returns.cumsum() - returns.cumsum().cummax()).min()
    
    return total_return, sharpe, max_drawdown

total_return, sharpe, max_dd = calculate_metrics(features["strategy_return"])

total_return, sharpe, max_dd


(np.float64(1.6546889774737052),
 np.float64(1.5379254080628262),
 np.float64(-0.18717230028030973))

## Strategy Notes

- EMA crossover is chosen for its simplicity and interpretability.
- Regime filtering helps avoid trades during sideways markets.
- The strategy is designed for demonstration and learning purposes.
- Transaction costs are ignored to focus on signal behavior.
